# Evaluation Metrics

we have used both NDCG and RMSE as evaluation metrics and the present code is to calculate NDCG.

In [152]:
import numpy as np
import pandas as pd
data = pd.read_csv("total_dataset.csv")
data=data[['_unit_id','rank']]
rank_dict ={}
count =0
for i in range(len(data)):
    unit_id = data.iloc[i]['_unit_id']
    rank = data.iloc[i]['rank']
    if(rank_dict.has_key(unit_id)):
        print("something wrong")
        break
    else:
        rank_dict[unit_id]=rank

In [158]:
def check_valid(rank):
    try: 
        int(rank)
        return True
    except ValueError:
        return False

In [157]:
def get_query(filename):
    results = np.load(filename)
    query_results ={}
    prev = results[0][0].encode('ascii')
    temp_list =[]
    for i in range(len(results)):
        if(results[i][0].encode('ascii')==prev):
            uid = int(results[i][1].encode('ascii'))
            rank = rank_dict[uid]
            temp2_list = results[i].tolist()
            if(check_valid(rank)):
                temp2_list.append(int(rank))
                temp_list.append(np.asarray(temp2_list))
        else:
            query_results[prev] = np.asarray(temp_list)
            temp_list=[]
            prev = results[i][0].encode('ascii')
            uid = int(results[i][1].encode('ascii'))
            rank = rank_dict[uid]
            temp2_list = results[i].tolist()
            if(check_valid(rank)):
                temp2_list.append(int(rank))
                temp_list.append(np.asarray(temp2_list))
    return query_results

In [156]:
def modify(temp_list):
    temp_out = {}
    for i in range(len(temp_list)):
        temp_lis = {}
        temp_lis['pid']=int(temp_list[i][1])
        temp_lis['actual'] = float(temp_list[i][2])
        temp_lis['predict'] = float(temp_list[i][3])
        key = int(temp_list[i][4])
        temp_out[key] = temp_lis
    return temp_out

In [155]:
def get_list(temp_out):
    temp_keys = temp_out.keys()
    temp_keys.sort()
    actual=[]
    predicted =[]
    for rank in temp_keys:
        temp_dict1 = temp_out[rank]
        actual.append(temp_dict1['actual'])
        predicted.append(temp_dict1['predict'])
    return actual,predicted

In [153]:
import math
def get_NDCG_helper(temp_dict,g_count):
    actual,predicted = get_list(temp_dict)
    ideal = []
    for item in actual:
        ideal.append(item)
    ideal.sort(reverse = True)
    IDCG_a = 0.0
    DCG_a =0.0
    count = 0
    for i in range(len(actual)):
        if(count==g_count):
            break
        if(i==0):
            DCG_a +=actual[0]
            IDCG_a +=ideal[0]
        else:
            DCG_a+=(actual[i]+0.0)/math.log(i+1,2)
            IDCG_a +=(ideal[i]+0.0)/math.log(i+1,2)
        count +=1
    NDCG_a = DCG_a/IDCG_a
    ideal = []
    for item in predicted:
        ideal.append(item)
    ideal.sort(reverse = True)
    IDCG_p = 0.0
    DCG_p=0.0
    count = 0
    for i in range(len(predicted)):
        if(count == g_count):
            break
        if(i==0):
            DCG_p +=predicted[0]
            IDCG_p +=ideal[0]
        else:
            DCG_p+=(predicted[i]+0.0)/math.log(i+1,2)
            IDCG_p +=(ideal[i]+0.0)/math.log(i+1,2)
        count +=1
    NDCG_p = DCG_p/IDCG_p
    return NDCG_a,NDCG_p

In [154]:
def get_NDCG(query_results,count=100):
    o_NDCG_a = 0.0
    o_NDCG_p =0.0
    for query in query_results.keys():
        temp_list = query_results[query]
        temp_dict= modify(temp_list)
        temp_NDCG_a,temp_NDCG_p = get_NDCG_helper(temp_dict,count)
        o_NDCG_a +=temp_NDCG_a
        o_NDCG_p +=temp_NDCG_p
    o_NDCG_a = o_NDCG_a/len(query_results)
    o_NDCG_p = o_NDCG_p/len(query_results)
    return o_NDCG_a,o_NDCG_p

In [161]:
def get_all_values(filename,count=100):
    query_results = get_query(filename)
    NDCG_a,NDCG_p = get_NDCG(query_results,count)
    return NDCG_a,NDCG_p

In [189]:
get_all_values('resDesc+ImageRaw.npy',5)

(0.9343724696225619, 0.9501738921722769)

In [190]:
get_all_values('resDesc+ImageRaw.npy',10)

(0.9603341473002783, 0.9709006797664881)